In [1]:
%pip install pandas nltk sklearn

  Using cached sklearn-0.0.post7.tar.gz (3.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [2]:
%pip install nltk

In [3]:
# Cell 2:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import pickle


In [4]:
%pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [5]:

data = pd.read_excel("REVIEW DATASET.xlsx")
data.head()


,FEEDBACK,RATING
0,But the staff was so horrible to us.,-1
1,"To be completely fair, the only redeeming fact...",1
2,"The food is uniformly exceptional, with a very...",1
3,"The food is uniformly exceptional, with a very...",1
4,"The food is uniformly exceptional, with a very...",0


In [6]:
# Cell 4:
# Download stopwords and Punkt Tokenizer
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Tokenize and remove stopwords
data['processed_reviews'] = data['FEEDBACK'].apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x.lower()) if word not in stop_words]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NAVEENKUMAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NAVEENKUMAR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Cell 5:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['processed_reviews'])
y = data['RATING']


In [8]:
# Cell 6:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Cell 7:
sentiment_model = LinearSVC()
sentiment_model.fit(X_train, y_train)


c:\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [10]:
# Cell 8:
y_pred = sentiment_model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          -1       0.70      0.55      0.62       183
           0       0.50      0.42      0.46       132
           1       0.79      0.90      0.84       424

    accuracy                           0.73       739
   macro avg       0.66      0.62      0.64       739
weighted avg       0.72      0.73      0.72       739



In [11]:
# Cell 9:
def extract_aspects(text):
    # For simplicity, this function assumes each noun in the text as an aspect.
    # In real-world scenarios, more complex algorithms or models like topic modeling or dependency parsing might be used.
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    aspects = [word for word, pos in pos_tags if pos in ['NN', 'NNS']]
    return aspects


In [12]:
# Cell 10:
def predict_aspect_sentiment(review):
    aspects = extract_aspects(review)
    results = {}
    for aspect in aspects:
        sentiment = sentiment_model.predict(vectorizer.transform([aspect]))[0]
        results[aspect] = "positive" if sentiment == 1 else "negative" if sentiment == -1 else "neutral"
    return results


In [13]:
# Cell 11:
with open('aspect_sentiment_model.pkl', 'wb') as model_file:
    pickle.dump(sentiment_model, model_file)
    
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)


In [15]:
# First, ensure that the necessary functions and models are loaded:
# If you've followed the steps in the previous .ipynb description, then this should be fine.
# If not, you might need to re-run the cells that define the necessary functions and load the models.

# Now, define a function to get custom input and show predictions:
def test_custom_input():
    # Get input from user
    review = input("Enter your review: ")

    # Predict the aspect sentiment using the provided function
    predictions = predict_aspect_sentiment(review)

    # Display the results
    for aspect, sentiment in predictions.items():
        print(f"Aspect: {aspect} - Sentiment: {sentiment}")

# Now, call this function to test
test_custom_input()


Aspect: taste - Sentiment: positive
Aspect: service - Sentiment: positive
